In [5]:
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine, MetaData, Table, Column, ForeignKey
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
engine = create_engine("sqlite:///Database/movielens.db", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
def sqlalchemy_mapping_init(database):
    

NameError: name 'Links' is not defined